<a href="https://colab.research.google.com/github/1kaiser/test2023/blob/main/GoogleEarthEngine_AnySatellite_Data_Download.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install the library
!pip -q install FireHR==0.1.2 pyhdf==0.10.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.1/149.1 KB 2.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.3/193.3 KB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.1/64.1 KB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 57.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.8/776.8 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 62.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.0/56.0 KB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 74.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 KB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 59.1

In [2]:
# Authenticate to use Google Earth Engine API
import ee
ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=UQpPJAQCKv5ExxvGiPDvWZYRKwVQY7deOc9CmTPA_jg&tc=axhHgoVStA6AgMVYq264l3Bw7knkQPQQG_gLHt1Vu98&cc=7GrLpaHZGjboeZDpcf64ONFjjYKcxEfvH__Aj0H93hY

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AWtgzh5hFxkViNzKi7Rm6C4VfTb0639_bpVwmckyzHUDsWPmIXyNjAY5F4A

Successfully saved authorization token.


In [ ]:
!rm -r /content/data

##**COPERNICUS/S2**

In [6]:
from pathlib import Path
from FireHR.data import *

# Bounding box coordinates
left   = 75.979728
right  = 77.866667
bottom = 31.453599
top    = 32.416667

path_save   = Path('data')
products    = ["COPERNICUS/S2"]  # Product id in google earth engine
bands       = ['B4', 'B3', 'B2'] # Red, Green, Blue

R = RegionST(name         = 'TeslaGigaBerlin', 
             bbox         = [left,bottom,right,top], 
             scale_meters = 10, 
             time_start   = '2021-03-01', 
             time_end     = '2021-04-25')

# Download time series
# download_data_ts(R, products, bands, path_save)

time_window = R.times[0], R.times[-1]

# Download median composite of the 3 least cloudy images within the time_window
download_data(R, time_window, products, bands, path_save, use_least_cloudy=3, show_progress=True)

#download_data_ts(R, products, bands, path_save, show_progress=True)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from banet.data import open_tif

brightness = 3
im = np.concatenate([open_tif(f'data/download.{b}.tif').read() for b in bands])
im = im.transpose(1,2,0).astype(np.float32)/10000
plt.imshow(brightness*im)

##**GEDI**

In [ ]:
from pathlib import Path
from FireHR.data import *

# Bounding box coordinates
left   = 75.979728
right  = 77.866667
bottom = 31.453599
top    = 32.416667

path_save   = Path('data')
products    = ["LARSE/GEDI/GEDI02_A_002_MONTHLY"]  # Product id in google earth engine
bands       = ['rh98', 'rh38', 'rh10'] # Red, Green, Blue

R = RegionST(name         = 'TeslaGigaBerlin', 
             bbox         = [left,bottom,right,top], 
             scale_meters = 10, 
             time_start   = '2020-03-01', 
             time_end     = '2021-07-25')

# Download time series
# download_data_ts(R, products, bands, path_save)

time_window = R.times[0], R.times[-1]

# Download median composite of the 3 least cloudy images within the time_window
download_data(R, time_window, products, bands, path_save, use_least_cloudy=3, show_progress=True)

#download_data_ts(R, products, bands, path_save, show_progress=True)